In [1]:
import os
from PIL import Image, ImageChops, ImageEnhance

In [3]:
def apply_ela(input_path, output_path, quality=90):
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    for filename in os.listdir(input_path):
        img_path = os.path.join(input_path, filename)
        output_img_path = os.path.join(output_path, filename)

        if img_path.lower().endswith(('.jpg', '.jpeg')):
            # Open the image and convert it to RGB mode
            image = Image.open(img_path).convert('RGB')
            
            # Resave the image with the specified quality
            image.save('temp.jpg', 'JPEG', quality=quality)
            resaved = Image.open('temp.jpg')

            # Calculate the ELA (Error Level Analysis) image by taking the difference between the original and resaved image
            ela_image = ImageChops.difference(image, resaved)

            # Get the minimum and maximum pixel values in the ELA image
            band_values = ela_image.getextrema()
            max_value = max([val[1] for val in band_values])

            # If the maximum value is 0, set it to 1 to avoid division by zero
            if max_value == 0:
                max_value = 1

            # Scale the pixel values of the ELA image to the range [0, 255]
            scale = 255.0 / max_value
            ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)

            # Save the ELA image
            ela_image.save(output_img_path)

    os.remove("temp.jpg")

# Apply ELA to the CASIA2 dataset


In [5]:
apply_ela("dataset/CASIA2/CASIA2/Au", "processed/Au")
apply_ela("dataset/CASIA2/CASIA2/Tp", "processed/Tp")

In [26]:
#apply_ela("dataset/IMD2020", "processed/Tp")

In [6]:
import os
import pandas as pd
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Augmentation for Training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Data Augmentation for Validation (No Augmentation)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Load all file paths and labels
all_files = []
all_labels = []

for class_folder in ['Au', 'Tp']:
    class_path = os.path.join("processed", class_folder)
    class_label = 0 if class_folder == 'Au' else 1
    
    files = [os.path.join(class_path, file) for file in os.listdir(class_path)]
    labels = [class_label] * len(files)
    
    all_files.extend(files)
    all_labels.extend(labels)

# Convert numerical labels to strings
all_labels = list(map(str, all_labels))

# Split data into training and validation sets
train_files, validation_files, train_labels, validation_labels = train_test_split(
    all_files, all_labels, test_size=0.2, random_state=42, stratify=all_labels
)

# Train and validation generators
train_generator = train_datagen.flow_from_dataframe(
    pd.DataFrame({'filename': train_files, 'class': train_labels}),
    directory=None,  # Use absolute paths in the dataframe
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    color_mode='grayscale',
    shuffle=True
)

validation_generator = validation_datagen.flow_from_dataframe(
    pd.DataFrame({'filename': validation_files, 'class': validation_labels}),
    directory=None,  # Use absolute paths in the dataframe
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    color_mode='grayscale',
    shuffle=False  # No need to shuffle the validation set
)


Found 7600 validated image filenames belonging to 2 classes.
Found 1901 validated image filenames belonging to 2 classes.


C:\Users\Jaseem\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=forgery_detection_model.h5